# Unit 5 Data Wrangling

## 1. Clean up CSVs and Excel files before importing


1. Set date format to yyyy-mm-dd. 
2. Removed unnecessary columns and rows. 
3. Updated column names to be consistent across the dataset
4. Formated column names to be consistent - start with capital letter. 
5. Filled in all empty data with "NA" 


## 2. Read in data into dataframe.
#### 2.1. Annual or Quarterly data are resampled and forward filled to Monthly. 


## 3. Combine all dataframes into one dataframe
#### 3.1. Common key is RegionName and Date. 
Data sets: 
1. Sales Price - monthly by county
2. Unemployment data - monthly by county
3. Mortgage rate - National
4. DaysOnZillow - monthly by county
5. Population data - annual by county
6. Historical Housing Affordability Index - Monthly/Quarterly
7. Crime rate - Annual

#### 3.2. Final Dataframes:
(Note: Currently combined into 2 because any resampled dataframe does not combined with non-resampled dataframe). 
1. master_df - SalePrice, UnemployRate, Days-On-Zillow, Monthly-List-Zillow, Rate (mortgage rate)
2. master_df_2nd - Population, Afford_idx, Property_Crimes, Violent_Crimes

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib
import matplotlib.pyplot as pp

%matplotlib inline

## 1. Sale Price by County

#### Frequency: Monthly
#### Time range: 2008/03 - 2019 /05
#### No Data: NA

In [2]:
# Clean Sales Price 
# read in California counties' sales price excel. Set encoding

CA_sales_prices = pd.read_excel(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Zillow_Sale_Prices_County_2008_2015_cleaned.xlsx',encoding='latin-1')

In [3]:
CA_sales_prices.head()

,RegionID,RegionName,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,...,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05
0,3101,Los Angeles County,463600.0,450100.0,433600.0,420900.0,403800.0,388500.0,372200.0,360700.0,...,576400,581500,590900,598100,596600.0,597800.0,595700.0,602800.0,605800.0,605100.0
1,2841,San Diego County,403100.0,394400.0,380000.0,368000.0,352800.0,341100.0,331100.0,320900.0,...,556600,559200,560800,555700,553100.0,551600.0,555100.0,553100.0,558000.0,NaN
2,1286,Orange County,514100.0,505400.0,489400.0,471100.0,457000.0,444200.0,434300.0,422200.0,...,705400,710900,709300,706000,696600.0,695600.0,694900.0,698700.0,701400.0,699600.0
3,2832,Riverside County,316800.0,299400.0,282700.0,270500.0,258800.0,246700.0,236000.0,228200.0,...,365000,369800,371800,374900,369900.0,372500.0,372800.0,374200.0,371800.0,371600.0
4,3250,San Bernardino County,267700.0,255500.0,241400.0,232900.0,219500.0,211100.0,200700.0,194800.0,...,316500,317100,318500,316300,312600.0,313900.0,318400.0,323200.0,325000.0,329800.0


In [4]:
# Convert year-month to date instead: 2008-10 to 2008-10-01 and then to datetime type. 
# For my notes - see http://jonathansoma.com/lede/foundations/classes/pandas%20columns%20and%20functions/apply-a-function-to-every-row-in-a-pandas-dataframe/ 
# section: Use .apply with axis=1 to send every single row to a function
def set_first_of_month(row):
    datestring = row['year-month'] +'-01'
    return datetime.strptime(datestring, '%Y-%m-%d')

# select county data

def get_cleaned_county_sales_price(CA_sales_prices, county_name):

    county_sales_price = CA_sales_prices.loc[CA_sales_prices['RegionName'] == county_name]
    
    # Melt table to get year and month prices as rows. 
    alm_melt = county_sales_price.melt(id_vars=['RegionID', 'RegionName'], var_name='year-month', value_name='SalePrice')

    # set month and year columns and drop the intermedia 'year-month' column
    #alm_melt['Month'] = alm_melt['year-month'].apply(lambda s: int(s[5:7]))
    #alm_melt['Year'] = alm_melt['year-month'].apply(lambda s: int(s[0:4]))
    
    alm_melt['Date']=alm_melt.apply(set_first_of_month, axis=1)
    alm_melt.drop(['year-month', 'RegionID'],axis=1,inplace=True) 

    sales_price_final = alm_melt
    
    return sales_price_final


In [5]:
# Get cleaned sales price data for Alameda county
alameda_county_name = 'Alameda County'
alameda_sales_price_final = get_cleaned_county_sales_price(CA_sales_prices, alameda_county_name)



In [6]:
# Get cleaned sales price data for Sacramento county
sacramento_county_name = 'Sacramento County'
sacramento_sales_price_final = get_cleaned_county_sales_price(CA_sales_prices, sacramento_county_name)


In [7]:
# Get cleaned sales price data for Los Angeles county
LA_county_name = 'Los Angeles County'
LA_sales_price_final = get_cleaned_county_sales_price(CA_sales_prices, LA_county_name)

# Combine into one dataframe

frames = [alameda_sales_price_final, sacramento_sales_price_final, LA_sales_price_final]
master_df= pd.concat(frames)


## 2. Unemployment data - monthly by county

#### Frequency: Monthly
#### Time range: 1990/01 - 2019/05

In [8]:
# Read in Alameda counties' sales price excel. Set encoding

alameda_unemploy = pd.read_excel(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Unemployment_CAALAM1URN_CA_Alameda_County_cleaned.xlsx',encoding='latin-1')
# Update DATE to datetime type.
alameda_unemploy['DATE'] = pd.to_datetime(alameda_unemploy['DATE'])  

# Rename columns to be consistent
alameda_unemploy.rename(columns={'DATE':'Date',
                          'CAALAM1URN':'UnemployRate'}, 
                 inplace=True)

# Separate Date into Year and Month - all other monthly data will have separate Year and Month.
#alameda_unemploy['Month'] = alameda_unemploy['Date'].apply(lambda s: s.month)
#alameda_unemploy['Year'] = alameda_unemploy['Date'].apply(lambda s: s.year)

alameda_unemploy['RegionName'] = 'Alameda County' 

In [9]:
# Read in Scramento counties' sales price csv. Set encoding

sacramento_unemploy = pd.read_excel(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Unemployment_CASACR5URN_Sacramento County_cleaned.xlsx',encoding='latin-1')
# Update DATE to datetime type.
sacramento_unemploy['DATE'] = pd.to_datetime(sacramento_unemploy['DATE'])  
# Rename columns to be consistent
sacramento_unemploy.rename(columns={'DATE':'Date',
                          'CASACR5URN':'UnemployRate'}, 
                 inplace=True)

# Separate Date into Year and Month - all other monthly data will have separate Year and Month.
#sacramento_unemploy['Month'] = sacramento_unemploy['Date'].apply(lambda s: s.month)
#sacramento_unemploy['Year'] = sacramento_unemploy['Date'].apply(lambda s: s.year)
sacramento_unemploy['RegionName'] = 'Sacramento County' 

In [10]:
# Read in LA counties' sales price csv. Set encoding

LA_unemploy = pd.read_csv(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Unemployment_CALOSA7URN_LA County_cleaned.csv',encoding='latin-1')
# Update DATE to datetime type.
LA_unemploy['DATE'] = pd.to_datetime(LA_unemploy['DATE'])  
# Rename columns to be consistent
LA_unemploy.rename(columns={'DATE':'Date',
                          'CALOSA7URN':'UnemployRate'}, 
                 inplace=True)

# Separate Date into Year and Month - all other monthly data will have separate Year and Month.
#LA_unemploy['Month'] = LA_unemploy['Date'].apply(lambda s: s.month)
#LA_unemploy['Year'] = LA_unemploy['Date'].apply(lambda s: s.year)
LA_unemploy['RegionName'] = 'Sacramento County'
LA_unemploy.head()

# Combine into one dataframe

frames = [alameda_unemploy, sacramento_unemploy, LA_unemploy]
master_unemploy= pd.concat(frames)

# Merge sales prices and unemployment - monthly tables
master_df = pd.merge(master_df, master_unemploy, on=['RegionName','Date'], how='inner')


## 3. Mortgage rate - National

#### Frequency: Monthly
#### Range: 2010/01 - 2018/12
#### At National level, not at county level.

In [11]:
def set_first_of_month_year(row):
    datestring = str(int(row['Year']))+ '-'+ str(int(row['Month'])) +'-01'
    return datetime.strptime(datestring, '%Y-%m-%d')

mort_rt = pd.read_excel(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Cleaned_MortageRate_2010-2018.xlsx',  encoding='latin-1')

# Generate Date from Month and Year string.
mort_rt['Date']=mort_rt.apply(set_first_of_month_year, axis=1)
mort_rt.drop(['Year', 'Month', 'Pts'],axis=1,inplace=True) 

# Set RegionName, duplicate for each county
mort_rt['RegionName'] = 'Alameda County'
sac_mort_rt = mort_rt.copy()
la_mort_rt = mort_rt.copy()

sac_mort_rt['RegionName'] = 'Sacramento County'
la_mort_rt['RegionName'] = 'Los Angeles County'

# combine into one dataframe
frames = [mort_rt, sac_mort_rt, la_mort_rt]
master_mort_rt= pd.concat(frames)


# Merge into monthly tables
master_df = pd.merge(master_df, master_mort_rt, on=['RegionName','Date'], how='inner')



## 4. DaysOnZillow - monthly by county


In [12]:
days_on_z = pd.read_csv(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Zillow_DaysOnZillow_County_2010_2019_Cleaned.csv',  encoding='latin-1')

days_on_z.head()

,RegionName,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,...,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
0,Los Angeles County,117.0,125.0,116.0,103.0,84.0,91.0,86.0,89.0,91.0,...,58.0,58.0,62.0,67.0,69.0,74.0,85.0,88.0,69.0,64.0
1,San Diego County,103.0,108.0,95.0,103.0,88.0,88.0,74.0,80.0,94.0,...,54.0,52.0,58.0,67.0,67.5,74.5,83.0,82.0,62.0,57.0
2,Orange County,126.0,157.0,127.0,76.0,77.0,89.0,86.0,91.0,92.0,...,58.0,61.0,67.0,69.0,74.0,78.0,90.0,90.0,68.0,63.0
3,Riverside County,105.0,110.0,118.0,123.0,112.0,102.0,96.0,92.0,98.0,...,68.0,67.0,75.0,76.0,73.0,82.0,90.0,95.0,82.0,77.0
4,San Bernardino County,105.0,111.0,115.0,115.0,103.0,97.0,88.0,93.0,96.0,...,66.0,67.0,68.0,74.0,77.0,83.0,90.0,94.0,80.0,74.0


In [13]:

def get_cleaned_days_on_z(days_on_z, county_name):
    df = days_on_z.loc[days_on_z['RegionName'] == county_name]
    df_melt = df.melt(id_vars=['RegionName'], var_name='year-month', value_name='Days-On-Zillow')

    df_melt['Date']=df_melt.apply(set_first_of_month, axis=1)
    return df_melt

county_name='Alameda County' 
# Melt table to get year and month prices as rows. 
alameda_days_on_z = get_cleaned_days_on_z(days_on_z, county_name) 
alameda_days_on_z.head()

county_name='Sacramento County'
# Melt table to get year and month prices as rows. 
sacramento_days_on_z = get_cleaned_days_on_z(days_on_z, county_name) 
sacramento_days_on_z.head()

county_name='Los Angeles County'
# Melt table to get year and month prices as rows. 
LA_days_on_z = get_cleaned_days_on_z(days_on_z, county_name) 
LA_days_on_z.head()


frames = [alameda_days_on_z, sacramento_days_on_z, LA_days_on_z]
master_days= pd.concat(frames)

master_days.drop(['year-month'],axis=1,inplace=True) 

master_days.tail()

# Merge sales prices and unemployment - monthly tables
master_df = pd.merge(master_df, master_days, on=['RegionName','Date'], how='inner')


## 5. Zillow - # of Monthly listing - Monthly by County


In [14]:

monthly_listing_z = pd.read_csv(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Zillow_MonthlyListings_SSA_AllHomes_County_2013_2019_cleaned.csv',  encoding='latin-1')

monthly_listing_z.head()

def get_cleaned_monthly_list_on_z(monthly_z, county_name):
    df = monthly_z.loc[monthly_z['RegionName'] == county_name]
    df_melt = df.melt(id_vars=['RegionName'], var_name='year-month', value_name='Monthly-List-Zillow')

    df_melt['Date']=df_melt.apply(set_first_of_month, axis=1)
    return df_melt

county_name='Alameda County' 

# Melt table to get year and month prices as rows. 
alameda_monthly_z = get_cleaned_monthly_list_on_z(monthly_listing_z, county_name) 
alameda_monthly_z.head()

county_name='Sacramento County'
# Melt table to get year and month prices as rows. 
sacramento_monthly_z = get_cleaned_monthly_list_on_z(monthly_listing_z, county_name) 
sacramento_monthly_z.head()

county_name='Los Angeles County'
# Melt table to get year and month prices as rows. 
LA_monthly_z = get_cleaned_monthly_list_on_z(monthly_listing_z, county_name) 
LA_monthly_z.head()


frames = [alameda_monthly_z, sacramento_monthly_z, LA_monthly_z]
master_monthly_z= pd.concat(frames)

master_monthly_z.drop(['year-month'],axis=1,inplace=True) 

master_monthly_z.tail()


,RegionName,Monthly-List-Zillow,Date
72,Los Angeles County,19999.0,2019-01-01
73,Los Angeles County,19809.0,2019-02-01
74,Los Angeles County,19336.0,2019-03-01
75,Los Angeles County,19378.0,2019-04-01
76,Los Angeles County,19647.0,2019-05-01


In [15]:
# Merge sales prices and unemployment - monthly tables
master_df = pd.merge(master_df, master_monthly_z, on=['RegionName','Date'], how='inner')


## 5. Population data - annual by county
   - Removed unnecessary columns.
   - Updated column title to date format
    


In [16]:
pop_data = pd.read_csv(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\Population_PEP_2018_PEPANNRES_with_ann_cleaned_1.csv',  encoding='latin-1')

pop_data.head()

def get_cleaned_pop(pop_data, county_name):
    # Get county specific data
    df = pop_data.loc[pop_data['Geography'] == county_name]
    
    # Melt the data
    df_melt = df.melt(id_vars=['Geography'], var_name='Date', value_name='Population')
    df_melt.rename(columns={'Geography':'RegionName'}, inplace=True)
    
    # set datetime index to convert data from annual to monthly
    df_melt['Date'] = pd.to_datetime(df_melt['Date'])
    df_melt.set_index('Date', inplace=True)
    df_melt_monthly = df_melt.resample('M').ffill().reset_index()
    
    return df_melt_monthly

county_name='Alameda County' 

# Melt table to get year and month prices as rows. 
alameda_pop = get_cleaned_pop(pop_data, county_name) 

county_name='Sacramento County'
sacramento_pop = get_cleaned_pop(pop_data, county_name) 

county_name='Los Angeles County'
LA_pop = get_cleaned_pop(pop_data, county_name) 

frames = [alameda_pop, sacramento_pop, LA_pop]
master_pop= pd.concat(frames) 


# not merged to master_df yet. Having trouble with combining dataframes by date field. - see v3.

### 6. Historical Housing Affordability Index - Quarterly


In [17]:
afford_idx = pd.read_excel(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\2019-Q1 Historical Housing Affordability Index_CA.xlsx',encoding='latin-1')
afford_idx

# melt data
df_melt = afford_idx.melt(id_vars=['Date'], var_name='RegionName', value_name='Afford_idx')

def resample_afford_idx(afford_data, county_name):
    df = afford_data.loc[afford_data['RegionName'] == county_name]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df_monthly = df.resample('M').ffill().reset_index()
    return df_monthly

# Resample Alameda data 
county_name='Alameda County' 
alameda_afford = resample_afford_idx(df_melt, county_name) 

# Resample Sacramento data 
county_name='Sacramento County' 
sac_afford = resample_afford_idx(df_melt, county_name) 

# Resample LA data 
county_name='Los Angeles County' 
la_afford = resample_afford_idx(df_melt, county_name) 

# Concatenate
frames = [alameda_afford, sac_afford, la_afford]
master_afford_idx= pd.concat(frames) 


C:\Users\yulmee\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
master_df_2nd = pd.merge(master_pop, master_afford_idx, on=['RegionName','Date'], how='inner')


### 7. Crime rate - Annual - State level


In [19]:
crime_data = pd.read_csv(r'C:\Users\yulmee\Documents\Yul-Mee\Springboard\Capstone 1\dataset\cleaned\State of California Department of Justice - Crimes_Annual.csv',  encoding='latin-1')

# melt data
crime_data_melt = crime_data.melt(id_vars=['Crime_Type'], var_name='Date', value_name='Crimes')

# pivot
crime_data_melt = crime_data_melt.pivot(index='Date', columns='Crime_Type').reset_index()

# flatten after pivot
crime_data_melt.columns = crime_data_melt.columns.droplevel(0)

# add new column RegionName
crime_data_melt['RegionName'] ='Alameda County'

# rename the columns
crime_data_melt.columns = ['Date', 'Property_Crimes', 'Violent_Crimes', 'RegionName']

# change Date field to datetime field
crime_data_melt.Date = pd.to_datetime(crime_data_melt.Date)


crime_data_melt.set_index('Date', inplace=True)
crime_data_melt_monthly = crime_data_melt.resample('M').ffill().reset_index()


# replicate dataframes for each county - deep copy
crime_data_melt_monthly_2 = crime_data_melt_monthly.copy()
crime_data_melt_monthly_2['RegionName'] = 'Sacramento County'

crime_data_melt_monthly_3 = crime_data_melt_monthly.copy()
crime_data_melt_monthly_3['RegionName'] = 'Los Angeles County'

# concat 3 datframes into master df
frames = [crime_data_melt_monthly, crime_data_melt_monthly_2, crime_data_melt_monthly_3]
crime_data_master= pd.concat(frames) 
 



In [20]:
master_df_2nd = pd.merge(master_df_2nd, crime_data_master, on=['RegionName','Date'], how='inner')

### 8. Merge Datasets into one master

In [45]:
master_df1 = master_df.copy()
master_df_2nd1 = master_df_2nd.copy()

# set date to first of the month
def set_to_first_of_month(row):
    return row['Date'].replace(day=1)

# set 2nd master set date to 1st of the month
master_df_2nd1['Date'] = master_df_2nd1.apply(set_to_first_of_month, axis=1)

# Merge 2 master datasets to one by regionname and date
master_df_master = pd.merge(master_df_2nd1, master_df1, on=['RegionName','Date'], how='inner')

In [47]:
master_df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 11 columns):
Date                   183 non-null datetime64[ns]
RegionName             183 non-null object
Population             183 non-null int64
Afford_idx             183 non-null float64
Property_Crimes        183 non-null object
Violent_Crimes         183 non-null object
SalePrice              183 non-null float64
UnemployRate           183 non-null float64
Rate                   183 non-null float64
Days-On-Zillow         183 non-null float64
Monthly-List-Zillow    183 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(1), object(3)
memory usage: 17.2+ KB
